# Bird Species Classier 
Built using the [Kaggle dataset](https://www.kaggle.com/gpiosenka/100-bird-species) by [Gerald Piosenka](https://www.kaggle.com/gpiosenka)

Project By: [Uzair](https://github.com/Uzair05)

## Download, extract and arrange data.

In [ ]:
#!pip install --upgrade kaggle
!kaggle datasets download -d gpiosenka/100-bird-species

In [2]:
import os 
import zipfile

try:
    os.mkdir("./data/")
except Exception as err:
    print(f"Error in creating data/:\t{err}")


localzip = "./100-bird-species.zip"
zip_ref = zipfile.ZipFile(localzip, 'r')
try:
    zip_ref.extractall('data/')
except Exception as err:
    print(f"Erorr in zip extraction:\t{err}")
finally:
    zip_ref.close()

## Handle Data
Create train and test data.

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [4]:
train_dir = "./data/train"
test_dir = "./data/test"
valid_dir = "./data/valid"

train_datagen = ImageDataGenerator(
    rescale = 1./255., 
    rotation_range = 40, 
    shear_range = 0.2,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True, 
    vertical_flip = True, 
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)
valid_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224,224), 
    batch_size=20, 
    class_mode="categorical", 
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(224,224), 
    class_mode="categorical", 
    batch_size=20
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir, 
    target_size=(224,224), 
    class_mode="categorical", 
    batch_size=20
)

Found 45980 images belonging to 315 classes.
Found 1575 images belonging to 315 classes.
Found 1575 images belonging to 315 classes.


## CNN Using Transfer Learning

In [12]:
from tensorflow.keras.applications import ResNet50
# Import pretrained wights
#local_weights = "./data/EfficientNetB3-birds-98.92.h5"
pretrained_model = ResNet50(
    input_shape=(224,224,3), 
    weights="imagenet", 
    include_top=False, 
)


In [13]:
pretrained_model.trainable = False

In [14]:
pretrained_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [18]:
last_output = pretrained_model.output

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(120, activation='relu')(x)
x = tf.keras.layers.Dense(120, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
#x = tf.keras.layers.Dense(32, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(315, activation='softmax')(x)

model = tf.keras.Model(pretrained_model.input, x)

model.compile(optimizer = "rmsprop", 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [19]:
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        accuracy = 0.95 # Percentage Accuracy.
        if(logs.get('accuracy') != None) and (logs.get('accuracy') >= accuracy): # Experiment with changing this value
          print(f"\nReached {accuracy*100}% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [20]:
history = model.fit(
    train_generator,
    epochs = 5, 
    #steps_per_epoch = 50, 
    validation_data = test_generator, 
    verbose = 1, 
    validation_steps = 10,
    callbacks=[callbacks]
)

Epoch 1/5
  10/2299 [..............................] - ETA: 1:03:06 - loss: 12.2050 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
history.history

In [ ]:
eval = model.evaluate(
    valid_generator,
    batch_size=20
)